# Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening

Yashad Samant

# Table of Contents
* [Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening](#Assignment-4:-Negamax-with-Alpha-Beta-Pruning-and-Iterative-Deepening)
    * [Overview](#Overview)
	* [Required Code](#Initial-Code)
        * [negamax](#negamax)
        * [negamaxIDS](#negamaxIDS)
        * [negamaxIDSab](#negamaxIDSab)
        * [EBF](#ebf)
        * [constructor TTT](#TTT)
        * [playgame functions](#playgame)
    * [Results](#Results)
	* [Grading](#Grading)
	* [Extra Credit](#Extra-Credit)


## Overview

In this assignment, we implement various variations of negamax algorithms like plain negamax, negamax with iterative depth search and negamax with alpha beta pruning. We also compare the results obtained from all the three algorithms to check for the best algorithm on the basis of efficiency. We compare the results based on the effective branching factor. 

## Required Code 

## NEGAMAX

In [1]:
def negamax(game, depthLeft):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        #print ('moves', move)
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

### NEGAMAXIDS

In [2]:
def negamaxIDS(game, maxDepth):
#   iterated the negamax algorithm till the max depth is reached or if the depth is 0.
    for depth in range(maxDepth):
        value, move = negamax(game, depth)
#   checked for none values because negamax returns none if values are not updated.
        if value is not None:      
            return [value, move]

### NEGAMAXIDSab

In [12]:
def negamaxIDSab(game, depthLeft):
    alpha = -float("inf")
    beta = float("inf")
    temp = 0.0
    bestScore = 0
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        #print ('moves', move)
        # Apply a move to current state
        temp = -alpha
        alpha = -beta
        beta = temp
        game.makeMove(move)

        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamaxIDSab(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if value>=beta:
            return [bestValue, bestMove]
        alpha = max(value,alpha)
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
        
    return [bestValue, bestMove]

In the code below,
* Initialize all the necessary variables and make copies of nNodes so that we don't use updated values.
* Check if the number of nodes and depth is less than 0 and return 0 if it is so. This signifies that we dont have any branching.
* Now to iterate to find the optimum branching factor, we first start by taking the average of 0 and number of nodes thus limiting ebf between them.
* Next, we apply formula to find potential ebf.
* Save the values in dummy variables initialized before and make changes to reduce the ebf value or increase it based on the obtained b.
* return final value

### EBF

In [4]:
def ebf(nNodes, depth, precision=0.01):
    minvalue=0
    maxvalue=nNodes
    maxrange=nNodes
    finalebf=1
    evalue=0
    final_ev=0
    #if depth > 0:
        #return 'failure'
    if nNodes<=0 or depth<0:
        return 0.0
  
    while minvalue<=maxvalue:
        b=(minvalue+maxvalue)/2
        if b!=1:
            evalue = (1-(b**(depth+1)))/(1-b)
        r=abs(evalue-nNodes)
        if r<maxrange:
            maxrange=r
            final_ev=evalue
            finalebf=b
        if r<precision:
            return finalebf
            
        if evalue<nNodes:
            minvalue=b
        if evalue>nNodes:
            maxvalue=b


We have defined a constructor, where the board and players are initialized. We also define functions to find the numerical locations, get necessary moves, check if one has won or not, change players and count number of nodes.

## TTT

In [5]:
class TTT(object):

    def __init__(self):
        self.board = [' ']*9
        self.player = 'X'
        if False:
            self.board = ['X', 'X', ' ', 'X', 'O', 'O', ' ', ' ', ' ']
            self.player = 'O'
        self.playerLookAHead = self.player
        self.movesExplored = 0 

    def locations(self, c):
        #print ([i for i, mark in enumerate(self.board) if mark == c])
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        #print (moves)
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None  ########################################################## CHANGED FROM -0.1

    def isOver(self):
        return self.getUtility() is not None

    def makeMove(self, move):
        self.movesExplored+=1
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
        
    def getNumberMovesExplored(self):
        return self.movesExplored

    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s

### PLAYGAME

In [6]:
def opponent(board):
    return board.index(' ')

def playGame(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        #score,move = negamax(game,depthLimit)
        score, move  = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
    print(game.getNumberMovesExplored())
    depth = 0
    x=0
    for location in game.board:
        if location != ' ':
            depth+=1
        if location == 'X':
            x+=1
    #print(depth)
    print('negamaxIDSab made', x, 'moves with ebf(',game.getNumberMovesExplored(),',',depth,') is', ebf(game.getNumberMovesExplored(),depth,precision=0.01))

In [7]:
def playGameIDS(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        #score,move = negamax(game,depthLimit)
        score, move  = negamaxIDS(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
    print(game.getNumberMovesExplored())      
    x=0
    depth = 0
    for location in game.board:
        if location != ' ':
            depth+=1
        if location == 'X':
            x+=1
    #print(depth)
    print('negamaxIDSab made', x, 'moves with ebf(',game.getNumberMovesExplored(),',',depth,') is', ebf(game.getNumberMovesExplored(),depth,precision=0.01))

In [8]:
def playGameIDSab(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        #score,move = negamax(game,depthLimit)
        score, move  = negamaxIDSab(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
    depth = 0
    x=0
    for location in game.board:
        if location != ' ':
            depth+=1
        if location == 'X':
            x+=1
    #print(depth)
    #print(game.getNumberMovesExplored())
    print('negamaxIDSab made', x, 'moves with ebf(',game.getNumberMovesExplored(),',',depth,') is', ebf(game.getNumberMovesExplored(),depth,precision=0.01))

In [15]:
print('_______________________________NEGAMAX_______________________________________')
print(' ')
game = TTT()
playGame(game,opponent,20)

_______________________________NEGAMAX_______________________________________
 
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 
558334
negamaxIDSab made 4 moves with ebf( 558334 , 7 ) is 6.464860577546915


In [14]:
print('_______________________________NEGAMAXIDS_______________________________________')
print(' ')
game = TTT()
playGameIDS(game,opponent,20)

_______________________________NEGAMAXIDS_______________________________________
 
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 
23338
negamaxIDSab made 3 moves with ebf( 23338 , 5 ) is 7.256620558095165


In [13]:
print('_______________________________NEGAMAXIDSab_______________________________________')
print(' ')
game = TTT()
playGameIDSab(game,opponent,20)

_______________________________NEGAMAXIDSab_______________________________________
 
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 2 for score 0
X|O|X
-----
 | | 
-----
 | | 
Player O to 3
X|O|X
-----
O| | 
-----
 | | 
Player X to 4 for score 0
X|O|X
-----
O|X| 
-----
 | | 
Player O to 5
X|O|X
-----
O|X|O
-----
 | | 
Player X to 6 for score 1
X|O|X
-----
O|X|O
-----
X| | 
negamaxIDSab made 4 moves with ebf( 1506 , 7 ) is 2.659096598625183


Here are some example results. <font color='red'>Updated October 8, 3:15pm </font>

In [16]:
game = TTT()
playGameIDSab(game, opponent, 10)

 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 2 for score 0
X|O|X
-----
 | | 
-----
 | | 
Player O to 3
X|O|X
-----
O| | 
-----
 | | 
Player X to 4 for score 0
X|O|X
-----
O|X| 
-----
 | | 
Player O to 5
X|O|X
-----
O|X|O
-----
 | | 
Player X to 6 for score 1
X|O|X
-----
O|X|O
-----
X| | 
negamaxIDSab made 4 moves with ebf( 1506 , 7 ) is 2.659096598625183


### RESULTS

* We can see that negamax itself explored more than 0.5 million nodes and took about 4 moves to get the output with a ebf around 6.
* IDS was much better negamax exploring just 23k nodes which is a respectable reduction to negamax. It also had an ebf of 7.
* IDSab was the best where it explored just 1k moves thus reducing time and ebf was around 2.

Thus, we can conclude that IDSab was the best method for tic tac toe. It might vary for other games. But when there are many possibilities to explore, IDSab is the clear choice as it reduces time significantly putting less pressure on the processor. 

## Grading

As always, download and extract from [A4grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A4grader.tar)

In [ ]:
%run -i A4grader.py


Testing negamax starting from ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of 1

--- 10/10 points. negamax correctly explored 124 states.

Testing negamax starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of -1 and move of 5

Testing negamaxIDS with max depth of 5, starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamaxIDS correctly returns value of -1 and move of 5

Testing negamaxIDSab starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 20/20 points. negamaxIDSab correctly returns value of -1 and move of 5

Testing playGame with opponent that always plays in highest numbered position.
 | | 
-----
 | | 
-----
 | | 
